In [ ]:
!git clone https://github.com/rajendrawarke/CarDamageDetection-SeverityPrediction.git
%cd CarDamageDetection-SeverityPrediction

Cloning into 'CarDamageDetection-SeverityPrediction'...
remote: Enumerating objects: 13699, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 13699 (delta 0), reused 0 (delta 0), pack-reused 13696 (from 2)
Receiving objects: 100% (13699/13699), 573.27 MiB | 52.71 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (15613/15613), done.
/content/CarDamageDetection-SeverityPrediction


In [4]:
!pip install -q ultralytics==8.0.210
!pip install -q roboflow
!pip install tensorflow==2.19.0
!pip install -q opencv-python-headless matplotlib seaborn scikit-learn pandas imgaug
!pip install -q scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 720.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 M

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 131.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import matplotlib
import seaborn as sns
import numpy as np
import ultralytics
from ultralytics import YOLO
import roboflow
import sklearn
import cv2
import os, glob, pprint
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
print('tensorflow : ',tf.__version__)
print('matplotlib : ',matplotlib.__version__)
print('sns : ',sns.__version__)
print('np : ',np.__version__)
print('sklearn : ',sklearn.__version__)
print('ultralytics : ',ultralytics.__version__)
print('roboflow : ',roboflow.__version__)
print('cv2 : ',cv2.__version__)

pprint.pprint(sorted(glob.glob("data/**", recursive=True))[:5])

tensorflow :  2.19.0
matplotlib :  3.10.0
sns :  0.13.2
np :  2.0.2
sklearn :  1.6.1
ultralytics :  8.0.210
roboflow :  1.2.11
cv2 :  4.10.0
[]


In [ ]:
## As data.yaml file is already defined in the roboflow dataset, we will not execute below code to create our own yaml file for training the yolo model
# data_yaml = """
# train: dataset/train/images
# val: dataset/valid/images
# nc: 3
# names: ['scratch','dent','glass_shatter']
# """
# with open("data.yaml","w") as f:
#     f.write(data_yaml.strip())
# print("Wrote data.yaml")

In [ ]:
import torch
def unsafe_torch_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return torch.serialization.orig_load(*args, **kwargs)

if not hasattr(torch.serialization, 'orig_load'):
    torch.serialization.orig_load = torch.load
    torch.load = unsafe_torch_load

In [ ]:
model = YOLO("yolov8n.pt")
model.train(data="datasets/data.yaml", epochs=15, imgsz=640, batch=8, name="car_damage_yolov8n")

100%|██████████| 6.23M/6.23M [00:00<00:00, 105MB/s]
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.210 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/data.yaml, epochs=15, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=car_damage_yolov8n, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x783055610cb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004

In [ ]:
# api keyfor wandb login : f08db23a1f25a5634bb41c3cf875470469d26c46

In [ ]:
results = model.predict(source="datasets/valid/images", show=True)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1795 /content/CarDamageDetection-SeverityPrediction/datasets/valid/images/0003_JPEG_jpg.rf.b2c63e8be2d1d024d89b85471710aad3.jpg: 640x640 (no detections), 105.7ms
image 2/1795 /content/CarDamageDetection-SeverityPrediction/datasets/valid/images/0004_JPEG_jpg.rf.88567e90361a79b8e4638ca19c6b4af7.jpg: 640x640 1 damaged, 109.6ms
image 3/1795 /content/CarDamageDetection-Severit

In [ ]:
metrics = model.val()

Ultralytics YOLOv8.0.210 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
val: Scanning /content/CarDamageDetection-SeverityPrediction/datasets/valid/labels.cache... 1795 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1795/1795 [00:00<?, ?it/s]
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 5, len(boxes) = 2304. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 225/225 [02:47<00:00,  1.34it/s]
                   all       1795       2304      0.488      0.382      0.405      0.251
Front-Windscreen-Damage       1795         13      0.332      0.462      0.383      0.308
      Headlight-Damage       1795         42      

In [ ]:
detector_path = "runs/detect/car_damage_yolov8n/weights/best.pt"
detector = YOLO(detector_path)

In [ ]:
os.makedirs("crops", exist_ok=True)
val_images = sorted(list(Path("datasets/valid/images").glob("*.*")))
crop_count = 0

In [ ]:
for img_path in val_images:
    res = detector.predict(source=str(img_path), imgsz=640, conf=0.25, verbose=False)
    r = res[0]
    img = cv2.imread(str(img_path))
    h,w = img.shape[:2]
    for i,box in enumerate(r.boxes):
        xyxy = box.xyxy[0].cpu().numpy()  # [x1,y1,x2,y2]
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        x1,y1,x2,y2 = map(int, xyxy)
        pad = 8
        x1, y1 = max(0,x1-pad), max(0,y1-pad)
        x2, y2 = min(w-1,x2+pad), min(h-1,y2+pad)
        crop = img[y1:y2, x1:x2]
        fname = f"crops/{img_path.stem}_box{i}_cls{cls}_conf{int(conf*100)}.jpg"
        cv2.imwrite(fname, crop)
        crop_count += 1

print(f"Saved {crop_count} crops to crops/")

Saved 2001 crops to crops/


In [ ]:
!zip -r yolo_evaluation.zip runs/detect/car_damage_yolov8n2

  adding: runs/detect/car_damage_yolov8n2/ (stored 0%)
  adding: runs/detect/car_damage_yolov8n2/val_batch0_labels.jpg (deflated 9%)
  adding: runs/detect/car_damage_yolov8n2/val_batch0_pred.jpg (deflated 9%)
  adding: runs/detect/car_damage_yolov8n2/confusion_matrix_normalized.png (deflated 18%)
  adding: runs/detect/car_damage_yolov8n2/val_batch1_pred.jpg (deflated 9%)
  adding: runs/detect/car_damage_yolov8n2/R_curve.png (deflated 10%)
  adding: runs/detect/car_damage_yolov8n2/confusion_matrix.png (deflated 18%)
  adding: runs/detect/car_damage_yolov8n2/P_curve.png (deflated 8%)
  adding: runs/detect/car_damage_yolov8n2/PR_curve.png (deflated 9%)
  adding: runs/detect/car_damage_yolov8n2/F1_curve.png (deflated 10%)
  adding: runs/detect/car_damage_yolov8n2/val_batch2_labels.jpg (deflated 8%)
  adding: runs/detect/car_damage_yolov8n2/val_batch2_pred.jpg (deflated 8%)
  adding: runs/detect/car_damage_yolov8n2/val_batch1_labels.jpg (deflated 9%)


In [ ]:
!zip -r crops.zip crops/

  adding: crops/ (stored 0%)
  adding: crops/812_jpeg_jpg.rf.eeda3316eef0548d6872aebc2ad3c5c7_box0_cls9_conf60.jpg (deflated 1%)
  adding: crops/smallscratches_original_20-jpg_218bb37e-493c-4b00-a70c-7a876551a936_jpg.rf.fd66851309cbee0b0aa3dda671f6ad39_box0_cls30_conf83.jpg (deflated 2%)
  adding: crops/train_original_D3-jpg_0e92a9ee-5d0b-456d-81db-7911fac0cd6c_jpg.rf.4b1570b47f53587b02cdcc66ed924cd4_box0_cls30_conf30.jpg (deflated 7%)
  adding: crops/237_jpeg_jpg.rf.aa0ccf0cee6d4e4776a815dfcfebd14e_box0_cls9_conf56.jpg (deflated 0%)
  adding: crops/Scratch_original_image45-jpg_55626d65-6bfa-49ff-972b-35996cef90be_jpg.rf.e0c883b522f57c24b4a310d7a6bd4b8e_box1_cls30_conf40.jpg (deflated 1%)
  adding: crops/window255_jpeg_jpg.rf.e7046334d2e7a92a1f478c00e1512536_box0_cls18_conf46.jpg (deflated 0%)
  adding: crops/453_jpeg_jpg.rf.a9cb756b8101985b0945df10a06c5c6b_box1_cls9_conf42.jpg (deflated 1%)
  adding: crops/car-accident-damaged-doors-side-body-close-up-dented-silver-132527292_jpg.rf.a1

In [ ]:
!zip -r runs.zip runs/

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/car_damage_yolov8n/ (stored 0%)
  adding: runs/detect/car_damage_yolov8n/val_batch0_labels.jpg (deflated 7%)
  adding: runs/detect/car_damage_yolov8n/val_batch0_pred.jpg (deflated 7%)
  adding: runs/detect/car_damage_yolov8n/train_batch1.jpg (deflated 2%)
  adding: runs/detect/car_damage_yolov8n/events.out.tfevents.1765164600.941b76d51feb.5029.0 (deflated 76%)
  adding: runs/detect/car_damage_yolov8n/confusion_matrix_normalized.png (deflated 18%)
  adding: runs/detect/car_damage_yolov8n/val_batch1_pred.jpg (deflated 7%)
  adding: runs/detect/car_damage_yolov8n/R_curve.png (deflated 10%)
  adding: runs/detect/car_damage_yolov8n/labels_correlogram.jpg (deflated 30%)
  adding: runs/detect/car_damage_yolov8n/confusion_matrix.png (deflated 18%)
  adding: runs/detect/car_damage_yolov8n/train_batch3490.jpg (deflated 4%)
  adding: runs/detect/car_damage_yolov8n/labels.jpg (deflated 19%)
  adding: runs/detect/c

In [1]:
# Create severity dataset
# image path,severity

In [2]:
!unzip crops.zip

Archive:  crops.zip
   creating: crops/
  inflating: crops/812_jpeg_jpg.rf.eeda3316eef0548d6872aebc2ad3c5c7_box0_cls9_conf60.jpg  
  inflating: crops/smallscratches_original_20-jpg_218bb37e-493c-4b00-a70c-7a876551a936_jpg.rf.fd66851309cbee0b0aa3dda671f6ad39_box0_cls30_conf83.jpg  
  inflating: crops/train_original_D3-jpg_0e92a9ee-5d0b-456d-81db-7911fac0cd6c_jpg.rf.4b1570b47f53587b02cdcc66ed924cd4_box0_cls30_conf30.jpg  
  inflating: crops/237_jpeg_jpg.rf.aa0ccf0cee6d4e4776a815dfcfebd14e_box0_cls9_conf56.jpg  
  inflating: crops/Scratch_original_image45-jpg_55626d65-6bfa-49ff-972b-35996cef90be_jpg.rf.e0c883b522f57c24b4a310d7a6bd4b8e_box1_cls30_conf40.jpg  
  inflating: crops/window255_jpeg_jpg.rf.e7046334d2e7a92a1f478c00e1512536_box0_cls18_conf46.jpg  
  inflating: crops/453_jpeg_jpg.rf.a9cb756b8101985b0945df10a06c5c6b_box1_cls9_conf42.jpg  
  inflating: crops/car-accident-damaged-doors-side-body-close-up-dented-silver-132527292_jpg.rf.a11e9fee37916c8316b9d11bcb149390_box1_cls22_conf90.

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

csv_file_path = 'severity_labels.csv'
data = pd.read_csv(csv_file_path)

image_folder_path = 'crops/'

train_df, val_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 1600 validated image filenames belonging to 3 classes.
Found 401 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="image path". These filename(s) will be ignored.
  warnings.warn(


In [41]:
from sklearn.utils import class_weight
class_labels = data['label'].values
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

Class weights: {0: np.float64(0.4442964935641367), 1: np.float64(1.5701960784313727), 2: np.float64(8.897777777777778)}


In [47]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define the number of classes
num_classes = 3

# Load the EfficientNetB0 base model
base = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base.trainable = False  # Freeze the base layers

# Define the model architecture
inputs = layers.Input(shape=(224, 224, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',  # Use 'categorical_crossentropy' if labels are one-hot encoded
              metrics=['accuracy'])
model.summary()

# Set up callbacks for training
callbacks = [
    ModelCheckpoint("best_severity_model.h5", monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

# Fit the model using the data generators
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=15,
                    callbacks=callbacks)
                    # class_weight=class_weight_dict)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,378,278 (16.70 MB)

 Trainable params: 328,707 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6128 - loss: 0.8448

50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.6147 - loss: 0.8427 - val_accuracy: 0.7506 - val_loss: 0.6683 - learning_rate: 1.0000e-04
Epoch 2/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.7566 - loss: 0.6905 - val_accuracy: 0.7506 - val_loss: 0.6670 - learning_rate: 1.0000e-04
Epoch 3/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.7346 - loss: 0.7035 - val_accuracy: 0.7506 - val_loss: 0.6664 - learning_rate: 1.0000e-04
Epoch 4/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.7521 - loss: 0.6775 - val_accuracy: 0.7506 - val_loss: 0.6660 - learning_rate: 1.0000e-04
Epoch 5/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.7480 - loss: 0.6820 - val_accuracy: 0.7506 - val_loss: 0.6664 - learning_rate: 1.0000e-04
Epoch 6/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.7511 - loss: 0.6773 - val_accuracy: 0.7506 - val_loss: 0.6655 - learning_rate: 1.0000e-04
Epoch 7/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.7466 - loss: 0.6890 - v

In [48]:
val_loss, val_acc = model.evaluate(val_generator)
print("Val loss:", val_loss, "Val acc:", val_acc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 941ms/step - accuracy: 0.7577 - loss: 0.6492
Val loss: 0.6683326959609985 Val acc: 0.7506234645843506


In [49]:
y_pred_prob = model.predict(val_generator)
y_true=val_generator.classes
y_pred=np.argmax(y_pred_prob,axis=1)

13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step


In [50]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred, target_names=['minor','moderate','severe']))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       minor       0.75      1.00      0.86       301
    moderate       0.00      0.00      0.00        85
      severe       0.00      0.00      0.00        15

    accuracy                           0.75       401
   macro avg       0.25      0.33      0.29       401
weighted avg       0.56      0.75      0.64       401

[[301   0   0]
 [ 85   0   0]
 [ 15   0   0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont

from ultralytics import YOLO
detector = YOLO("best.pt")
severity_model = tf.keras.models.load_model("best_severity_model.h5")

def infer_and_visualize(img_path, conf_thresh=0.25):
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    res = detector.predict(source=str(img_path), imgsz=640, conf=conf_thresh, verbose=False)
    r = res[0]
    pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil)
    for box in r.boxes:
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        x1,y1,x2,y2 = xyxy
        pad = 8
        x1, y1 = max(0,x1-pad), max(0,y1-pad)
        x2, y2 = min(w-1,x2+pad), min(h-1,y2+pad)
        crop = img[y1:y2, x1:x2]
        crop_resized = cv2.resize(crop, (224,224))/255.0
        pred = severity_model.predict(np.expand_dims(crop_resized, 0))
        severity = ['minor','moderate','severe'][int(np.argmax(pred))]
        draw.rectangle([x1,y1,x2,y2], outline="red", width=3)
        draw.text((x1, max(0,y1-18)), f"{severity} {conf:.2f}", fill="yellow")
    display(pil)

infer_and_visualize("dataset/valid/images/example.jpg", conf_thresh=0.3)